In [10]:
from bs4 import BeautifulSoup as bs
import urllib2
import xml.etree.ElementTree as ET

In [248]:
page = urllib2.urlopen('http://archiveofourown.org/works/5176751')
soup = bs(page)

In [234]:
# print soup.body.prettify()

In [235]:
# print soup.body.get_text()

In [236]:
# for tag in soup.body.find_all('div'):
#     print tag.text

In [278]:
# tag = soup.body.find('div')
# print tag.text

In [249]:
#This one finds the main content, not considering multi-chapter and adult contents
tag = soup.body.find('div')
text = str(tag.text.encode('utf-8'))
text1 = text.split('Work Header')[1]
text2 = str(text1).split('Actions')[0]

In [381]:
#This seems more secure.
contents = re.findall('Work Header+.+Actions',text,flags=re.DOTALL)
contents = re.sub('\\n','',str(contents))

In [382]:
print contents

['Work Header\n\n\n\n              Rating:\n          \n\n\nTeen And Up Audiences\n\n\n\nArchive Warning:\n          \n\n\nNo Archive Warnings Apply\n\n\n\n              Category:\n          \n\n\nM/M\n\n\n\n              Fandoms:\n          \n\n\nHannibal (TV)Hannibal Lecter Series - All Media Types\n\n\n\n              Relationship:\n          \n\n\nWill Graham/Hannibal Lecter\n\n\n\n              Characters:\n          \n\n\nWill GrahamHannibal Lecter\n\n\n\n              Additional Tags:\n          \n\n\nDrinking Cum\n\n\n\n        Language:\n      \n\n        English\n      \nStats:\n\n\nPublished:2015-11-09Words:554Chapters:1/1Kudos:11Hits:162\n\n\n\n\n\n\n\n      A Delicious Drink\n    \n\nLoony Puppeteer (rea40)\n\n\nSummary:\n\nWill offers to get them something to drink. He brings Hannibal his favorite drink of all, in a martini glass.\n\n\n\nNotes:\n\nDisclaimer: I do not own the Hannibal characters or story in any way. I am making no money off of this.Hope you enjoy!\n\n\n\n

In [384]:
import re
from itertools import izip

# s = 'Title: Example: of a title Date: November 23 1234 Other: Other information'

keys = ['Rating', 'Archive Warning', 'Other']
pattern = re.compile('({})\s+'.format(':|'.join(keys)))

print dict(izip(*[(i.strip() for i in (pattern.split(contents)) if i)]*2))

{}


In [374]:
for i in str(contents).split('\\n'):
    print i

['Work Header



              Rating:
          


Teen And Up Audiences



Archive Warning:
          


No Archive Warnings Apply



              Category:
          


M/M



              Fandoms:
          


Hannibal (TV)Hannibal Lecter Series - All Media Types



              Relationship:
          


Will Graham/Hannibal Lecter



              Characters:
          


Will GrahamHannibal Lecter



              Additional Tags:
          


Drinking Cum



        Language:
      

        English
      
Stats:


Published:2015-11-09Words:554Chapters:1/1Kudos:11Hits:162







      A Delicious Drink
    

Loony Puppeteer (rea40)


Summary:

Will offers to get them something to drink. He brings Hannibal his favorite drink of all, in a martini glass.



Notes:

Disclaimer: I do not own the Hannibal characters or story in any way. I am making no money off of this.Hope you enjoy!






Work Text:
Will and Hannibal were sitting comfortably in the den. They sat together on the 

In [363]:
Rating = re.findall('Rating:\\n*\t*.*\n', str(contents))
Rating

[]

In [279]:
# tag = soup.body.find_all('div')[0]
# print tag

In [258]:
t = soup.body.find(string = '<body class="logged-out">')
print t

None


In [252]:
text2

'\n\n\n\n              Rating:\n          \n\n\nTeen And Up Audiences\n\n\n\nArchive Warning:\n          \n\n\nNo Archive Warnings Apply\n\n\n\n              Category:\n          \n\n\nM/M\n\n\n\n              Fandoms:\n          \n\n\nHannibal (TV)Hannibal Lecter Series - All Media Types\n\n\n\n              Relationship:\n          \n\n\nWill Graham/Hannibal Lecter\n\n\n\n              Characters:\n          \n\n\nWill GrahamHannibal Lecter\n\n\n\n              Additional Tags:\n          \n\n\nDrinking Cum\n\n\n\n        Language:\n      \n\n        English\n      \nStats:\n\n\nPublished:2015-11-09Words:554Chapters:1/1Kudos:11Hits:162\n\n\n\n\n\n\n\n      A Delicious Drink\n    \n\nLoony Puppeteer (rea40)\n\n\nSummary:\n\nWill offers to get them something to drink. He brings Hannibal his favorite drink of all, in a martini glass.\n\n\n\nNotes:\n\nDisclaimer: I do not own the Hannibal characters or story in any way. I am making no money off of this.Hope you enjoy!\n\n\n\n\n\n\nWork T

In [251]:
# for tag in soup.body.find_all(True):
#     print tag.name

In [50]:
for string in soup.body.strings:
    if string != '\n':
        print(repr(string))

u'Main Content'
u"\n    While we've done our best to make the core functionality of this site accessible without javascript, it will work better with it enabled. Please consider turning it on!\n  "
u'Archive of Our Own'
u' beta'
u'Log In'
u'User name:'
u'Password:'
u'Remember Me'
u'Forgot password?'
u'Get an Invitation'
u'Site Navigation'
u'Fandoms'
u'All Fandoms'
u'Anime & Manga'
u'Books & Literature'
u'Cartoons & Comics & Graphic Novels'
u'Celebrities & Real People'
u'Movies'
u'Music & Bands'
u'Other Media'
u'Theater'
u'TV Shows'
u'Video Games'
u'Uncategorized Fandoms'
u'Browse'
u'Works'
u'Bookmarks'
u'Tags'
u'Collections'
u'Search'
u'Works'
u'Bookmarks'
u'Tags'
u'People'
u'About'
u'About Us'
u'News'
u'FAQ'
u'Wrangling Guidelines'
u'Donate or Volunteer'
u'Search Works'
u'Work Search:'
u'tip: arthur merlin words>1000 sort:hits'
u'\xa0'
u'Skip header'
u'Actions'
u'Comments'
u'Share'
u'Copy and paste the following code to link back to this work ('
u'CTRL A'
u'/'
u'CMD A'
u' will select 

In [23]:
for link in soup.find_all('a'):
    print(link.get('href'))

#main
/
/user_sessions/new
/passwords/new
/invite_requests
/menu/fandoms
/media
/media/Anime%20*a*%20Manga/fandoms
/media/Books%20*a*%20Literature/fandoms
/media/Cartoons%20*a*%20Comics%20*a*%20Graphic%20Novels/fandoms
/media/Celebrities%20*a*%20Real%20People/fandoms
/media/Movies/fandoms
/media/Music%20*a*%20Bands/fandoms
/media/Other%20Media/fandoms
/media/Theater/fandoms
/media/TV%20Shows/fandoms
/media/Video%20Games/fandoms
/media/Uncategorized%20Fandoms/fandoms
/menu/browse
/works
/bookmarks
/tags
/collections
/menu/search
/works/search
/bookmarks/search
/tags/search
/people/search
/menu/about
/about
/admin_posts
/faq
/wrangling_guidelines
/donate
None
#work
#comments
#share
http://twitter.com/share
http://www.tumblr.com/share/link?url=http%3A%2F%2Farchiveofourown.org%2Fworks%2F5176751&name=A%20Delicious%20Drink%20%28554%20words%29%20by%20Loony%20Puppeteer%20%5BAO3%5D&description=Chapters%3A%201%2F1%3Cbr%20%2F%3EFandom%3A%20%3Ca%20href%3D%22http%3A%2F%2Farchiveofourown.org%2Ftags%